In [22]:
import Utilities
Utilities.getViT()
Utilities.getSquareImages()

Downloading...
From (original): https://drive.google.com/uc?id=1EZiws_atuzFLapKYyM9j2668UHSdtAs8
From (redirected): https://drive.google.com/uc?id=1EZiws_atuzFLapKYyM9j2668UHSdtAs8&confirm=t&uuid=64f64554-b203-421c-abb5-5a5e3f4a2af0
To: /Users/mawy/Desktop/saevision/vit.cpl
100%|██████████| 343M/343M [00:10<00:00, 32.5MB/s] 


[LOG] Downloaded ViT model to /Users/mawy/Desktop/saevision/vit.cpl
[LOG] Extracting Images-2.zip → /Users/mawy/Desktop/saevision/Square Images
[LOG] Extracting Images-3.zip → /Users/mawy/Desktop/saevision/Square Images
[LOG] Extracting Images-1.zip → /Users/mawy/Desktop/saevision/Square Images
[LOG] Extracting Images-4.zip → /Users/mawy/Desktop/saevision/Square Images
[LOG] Images ready in '/Users/mawy/Desktop/saevision/Square Images' (flattened)


In [ ]:
loader = Utilities.createImageDataloader(
    path="/Users/mawy/Desktop/saevision/Square Images",
    model_name="openai/clip-vit-base-patch32",
    batch_size=64,
    shuffle=True,
    drop_last=True,
)

In [21]:
import torch
model = torch.load("vit.cpl")